In [3]:
import igl
import pybcclean as pbc
import polyfempy as pf
import wildmeshing as wm
import meshplot as mp
import os
import numpy as np
from scipy.spatial import KDTree
import yaml
def barycenters(v, f):
    s = np.zeros((len(f),3),dtype='float64')
    for i, r in enumerate(s):
        s[i] = (v[f[i][0]]+v[f[i][1]]+v[f[i][2]])/3
    return s

def parse_feat(in_feat_file_name, out_file_body="feat"):
    parent_dir = os.path.dirname(in_feat_file_name)
    out_file_name = out_file_body+".dmat"
    out_file_rel_dir = os.path.join(parent_dir, out_file_name)
        # return the file path if the output file already exists
    with open(in_feat_file_name, 'r') as stream:
        try:
            yaml_dict=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    face_label_dict = {}
    count =0
    for label, surface in enumerate(yaml_dict["surfaces"]):
        for face_id in surface["face_indices"]:
            face_label_dict[face_id] = label
            count +=1

    with open(out_file_rel_dir, 'w') as f:
        f.write("1"+" "+str(count)+"\n")
        for item in sorted(face_label_dict.keys()):
            f.write("%s\n" % face_label_dict[item])
    return out_file_rel_dir

def separate_surfaces(v, f, fl):
    """
    :param v vertices
    :param f faces
    :param fl face labels
    perturb the surfaces according to the face labels
    :return v_dict, f_dict
    """
    v_dict = {}
    f_dict_temp = {}
    f_dict = {}
    count_dict = {}
    for fidx, lb in enumerate(fl):
        if lb not in f_dict_temp:
            f_dict_temp[lb] = np.zeros_like(f)
            f_dict_temp[lb][0,:]=f[fidx,:].copy()
            count_dict[lb] = 1
        else:
            f_dict_temp[lb][count_dict[lb],:]=f[fidx,:].copy()
            count_dict[lb] += 1
    for lb in f_dict_temp:
        f_dict_temp[lb] = f_dict_temp[lb][0:count_dict[lb],:]
        v_dict[lb], f_dict[lb], _, _= igl.remove_unreferenced(v,f_dict_temp[lb])
    return v_dict, f_dict

def perturb_and_union(v_dict, f_dict, eps):
    """
    :eps float to control the perturbation
    return new_v, new_f, new_fl
    """
    nv = 0 
    nf = 0
    for lb in f_dict:
        nv += len(v_dict[lb])
        nf += len(f_dict[lb])
    new_v = np.zeros((nv,3),dtype=float)
    new_f = np.zeros((nf,3), dtype=int)
    new_fl = np.zeros((nf,1), dtype=int)
    count_v = 0
    count_f =0
    for lb in f_dict:
        f_temp = f_dict[lb].copy()
        v_temp = v_dict[lb].copy()
        new_v[count_v:count_v+len(v_temp)]= v_temp + np.ones_like(v_temp) * np.random.uniform(-eps, eps)
        f_temp += np.ones_like(f_temp) * count_v
        count_v += len(v_temp)
        new_f[count_f:count_f+len(f_temp)]=f_temp
        new_fl[count_f:count_f+len(f_temp)]= np.ones((len(f_temp),1), dtype=int) *lb
        count_f += len(f_temp)
    return new_v, new_f, new_fl

def extract_surface(p, t):
    f_temp = igl.boundary_facets(t)
    v_s, f_s, _, _ = igl.remove_unreferenced(p, f_temp)
    return v_s, f_s


    

file_root = os.path.dirname("2/00000006_d4fe04f0f5f84b52bd4f10e4_trimesh_001.obj")
wm.tetrahedralize("2/00000006_d4fe04f0f5f84b52bd4f10e4_trimesh_001.obj", file_root+"/"+"bench.mesh", stop_quality=7)
fl_bench_file = parse_feat("2/00000006_d4fe04f0f5f84b52bd4f10e4_features_001.yml")
fl_bench = igl.read_dmat(fl_bench_file)
v_bench, f_bench = igl.read_triangle_mesh("2/00000006_d4fe04f0f5f84b52bd4f10e4_trimesh_001.obj")
v_ini, f_ini=igl.read_triangle_mesh(file_root+"/"+"bench.mesh__sf.obj")
prob_mat_ini, fl_ini_temp =pbc.project_face_labels(v_bench,f_bench.astype('int32'), fl_bench.astype('int32'),v_ini,f_ini.astype('int32'))
fl_ini = pbc.refine_labels(v_ini, f_ini.astype('int32'), prob_mat_ini, fl_ini_temp.astype('int32'),1)
# mp.plot(v_ini, f_ini, fl_ini_temp[:,0])
# mp.plot(v_ini, f_ini, fl_ini[:,0])
eps = 0.001
v_dict, f_dict =separate_surfaces(v_ini, f_ini, fl_ini[:,0])
v_bad, f_bad, fl_bad = perturb_and_union(v_dict, f_dict, eps)
mp.plot(v_bad, f_bad, fl_bad[:,0], shading={"wireframe":True})
eps_file_name = file_root+"/"+str(eps)+"pertb.obj"
igl.write_obj(eps_file_name, v_bad, f_bad)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000274…

True

In [5]:
mp.plot(v_bench, f_bench, fl_bench, shading={"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [8]:
eps = 0.1
v_dict, f_dict =separate_surfaces(v_ini, f_ini, fl_ini[:,0])
v_bad, f_bad, fl_bad = perturb_and_union(v_dict, f_dict, eps)
# mp.plot(v_bad, f_bad, fl_bad[:,0], shading={"wireframe":True})
eps_file_name = file_root+"/"+str(eps)+"pertb.obj"
igl.write_obj(eps_file_name, v_bad, f_bad)
igl.hausdorff(v_bench,f_bench, v_bad, f_bad)

0.1871525251619649

In [2]:
igl.write_obj("test.obj", v_bad, f_bad)

True

In [59]:
def build_label_dict(v, f, fl):
    """
    :param v vertices
    :param f faces
    :param fl face labels
    perturb the surfaces according to the face labels
    :return lebel_dict
    """
    v_dict = {}
    label_dict_temp = {}
    label_dict = {}
    count_dict = {}
    for fidx, lb in enumerate(fl):
        if lb not in label_dict_temp:
            label_dict_temp[lb] = np.zeros_like(f)
            label_dict_temp[lb][0,:]=f[fidx,:].copy()
            count_dict[lb] = 1
        else:
            label_dict_temp[lb][count_dict[lb],:]=f[fidx,:].copy()
            count_dict[lb] += 1
    for lb in label_dict_temp:
        label_dict[lb] = label_dict_temp[lb][0:count_dict[lb],:]
    return label_dict

def build_boundary_dict(label_dict):
    """
    :param fl face labels
    perturb the surfaces according to the face labels
    :return boundary_dict
    """
    boundary_dict = {}
    for lb in label_dict:
        boundary_dict[lb]=igl.boundary_facets(label_dict[lb])
    return boundary_dict

def build_node_dict(boundary_dict):
    node_dict= {}
    v_dict = {}
    for lb in boundary_dict:
        for line in boundary_dict[lb]:
            x = line[0]
            y = line[1]
            if x not in v_dict:
                v_dict[x]={}
            if y not in v_dict:
                v_dict[y]={}
            v_dict[x][lb]=1
            v_dict[y][lb]=1
    for v in v_dict:
        if len(v_dict[v])>2:
            for lb in v_dict[v]:
                if lb not in node_dict:
                    node_dict[lb]= []
                node_dict[lb].append(v)
    return node_dict

def select_vertices(v, indices):
    new_v = np.zeros((len(indices),3))
    for count,idx in enumerate(indices):
        new_v[count]= v[idx]
    return new_v
            

label_dict = build_label_dict(v_bench, f_bench, fl_bench)
boundary_dict=build_boundary_dict(label_dict)
p= mp.plot(v_bench,  boundary_dict[2],return_plot=True)
node_dict = build_node_dict(boundary_dict)
new_v = select_vertices(v_bench,node_dict[2])
p.add_points(new_v,shading={"point_color": "red", "point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 32.4…

1

In [60]:

lb=9
label_dict1 = build_label_dict(v_good, f_good, fl_good_cut[:,0])

boundary_dict1=build_boundary_dict(label_dict1)
p1= mp.plot(v_good,  boundary_dict1[lb],return_plot=True)
node_dict1 = build_node_dict(boundary_dict1)
new_v1 = select_vertices(v_good,node_dict1[lb])
p1.add_points(new_v1,shading={"point_color": "red", "point_size": 1})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.017742…

1

In [3]:
wm.tetrahedralize(eps_file_name, file_root+"/"+"out.mesh", stop_quality=7)
v_good, f_good = igl.read_triangle_mesh(file_root+"/"+"out.mesh__sf.obj")
print(f_good.shape, v_good.shape)
mp.plot(v_good, f_good, shading={"wireframe":True})

(6722, 3) (3361, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.005958…

In [4]:
def write_dmat(fl, out_file_body="feat"):
    out_file_name = out_file_body+".dmat"
    with open(out_file_name, 'w') as f:
        f.write("1"+" "+str(len(fl))+"\n")
        for i in range(len(fl)):
            f.write("%s\n" % fl[i][0])
    return out_file_name

In [5]:
dmat_file=write_dmat(fl_bad, out_file_body="test")

In [6]:
igl.write_obj("test_good.obj", v_good, f_good)

True

In [32]:
fl_bad = igl.read_dmat(dmat_file)
prob_mat, fl_good_proj = pbc.project_face_labels(v_bad, f_bad.astype('int32'), fl_bad.astype('int32'), v_good, f_good.astype('int32'))
fl_good_cut = pbc.refine_labels(v_good, f_good.astype('int32'), prob_mat, fl_good_proj.astype('int32'),1)
mp.plot(v_good, f_good, fl_good_proj[:,0], shading={"wireframe": True})
mp.plot(v_good, f_good, fl_good_cut[:,0], shading={"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.005958…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.005958…

In [36]:
bary_centers = barycenters(v_good, f_good)
kdt = KDTree(bary_centers)

bary_centers_ini = barycenters(v_ini, f_ini)
kdt_ini = KDTree(bary_centers_ini)

def sideset_proj(p):
    return fl_good_proj[kdt.query(p)[1]][0] + 1
def sideset_cut(p):
    return fl_good_cut[kdt.query(p)[1]][0] + 1
def sideset_ini(p):
    return fl_ini[kdt_ini.query(p)[1]][0] + 1

solver_ini = pf.Solver()
solver_ini.load_mesh_from_path(file_root+"/"+"bench.mesh")
solver_ini.set_boundary_side_set_from_bary(sideset_ini)
p_ini, t_ini, s_ini = solver_ini.get_boundary_sidesets()
mp.plot(p_ini, t_ini, s_ini, shading={"wireframe":True})

solver_proj = pf.Solver()
solver_proj.load_mesh_from_path(file_root+"/"+"out.mesh")
solver_proj.set_boundary_side_set_from_bary(sideset_proj)
p_proj, t_proj, s_proj = solver_proj.get_boundary_sidesets()
mp.plot(p_proj, t_proj, s_proj, shading={"wireframe":True})

solver_cut = pf.Solver()
solver_cut.load_mesh_from_path(file_root+"/"+"out.mesh")
solver_cut.set_boundary_side_set_from_bary(sideset_cut)
p_cut, t_cut, s_cut = solver_cut.get_boundary_sidesets()
mp.plot(p_cut, t_cut, s_cut, shading={"wireframe":True})


[2019-08-30 17:17:06.183] [polyfem] [info] Loading mesh...
[2019-08-30 17:17:06.183] [geogram] [info] Loading file 3/bench.mesh...
[2019-08-30 17:17:06.225] [geogram] [info] (FP64) nb_v:3656 nb_e:0 nb_f:7080 nb_b:0 tri:1 dim:3
[2019-08-30 17:17:06.225] [geogram] [info]  nb_tets:10901
[2019-08-30 17:17:06.225] [geogram] [info] Attributes on vertices: point[3]
[2019-08-30 17:17:06.378] [polyfem] [info] mesh bb min [-39.3665, -39.3688, -2.54], max [39.3629, 39.3645, 12.7]
[2019-08-30 17:17:06.379] [polyfem] [info]  took 0.196523s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001787…

[2019-08-30 17:17:07.826] [polyfem] [info] Loading mesh...
[2019-08-30 17:17:07.826] [geogram] [info] Loading file 3/out.mesh...
[2019-08-30 17:17:07.871] [geogram] [info] (FP64) nb_v:3471 nb_e:0 nb_f:6722 nb_b:0 tri:1 dim:3
[2019-08-30 17:17:07.871] [geogram] [info]  nb_tets:10331
[2019-08-30 17:17:07.871] [geogram] [info] Attributes on vertices: point[3]
[2019-08-30 17:17:08.037] [polyfem] [info] mesh bb min [-39.3668, -39.3429, -2.54088], max [39.3549, 39.3403, 12.7009]
[2019-08-30 17:17:08.038] [polyfem] [info]  took 0.211467s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.005958…

[2019-08-30 17:17:09.526] [polyfem] [info] Loading mesh...
[2019-08-30 17:17:09.526] [geogram] [info] Loading file 3/out.mesh...
[2019-08-30 17:17:09.575] [geogram] [info] (FP64) nb_v:3471 nb_e:0 nb_f:6722 nb_b:0 tri:1 dim:3
[2019-08-30 17:17:09.575] [geogram] [info]  nb_tets:10331
[2019-08-30 17:17:09.575] [geogram] [info] Attributes on vertices: point[3]
[2019-08-30 17:17:09.725] [polyfem] [info] mesh bb min [-39.3668, -39.3429, -2.54088], max [39.3549, 39.3403, 12.7009]
[2019-08-30 17:17:09.725] [polyfem] [info]  took 0.199019s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.005958…

In [55]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)


problem.set_displacement(5, [0, 0, 0])
problem.set_force(13, [0, 0, 0.1])

settings.set_problem(problem)
solver_proj.settings(settings)
solver_proj.solve()
p, t, d = solver_proj.get_sampled_solution()
m = np.linalg.norm(d, axis=1)

print(p.shape)
print(t.shape)
print(d.shape)
mp.plot(p+d, t, m)

[2019-08-30 17:23:55.590] [polyfem] [info] simplex_count: 	10331
[2019-08-30 17:23:55.590] [polyfem] [info] regular_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] regular_boundary_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] simple_singular_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] multi_singular_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] boundary_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] non_regular_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] undefined_count: 	0
[2019-08-30 17:23:55.590] [polyfem] [info] total count:	 10331
[2019-08-30 17:23:55.590] [polyfem] [info] Building isoparametric basis...
[2019-08-30 17:23:55.646] [polyfem] [info] Computing polygonal basis...
[2019-08-30 17:23:55.646] [polyfem] [info]  took 4.4881e-05s
[2019-08-30 17:23:55.652] [polyfem] [info] hmin: 0.4719

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.006153…

In [56]:
solver_cut.settings(settings)
solver_cut.solve()
p1, t1, d1 = solver_cut.get_sampled_solution()
m1 = np.linalg.norm(d1, axis=1)


mp.plot(p1+d1, t1, m1)
print((fl_good_proj).max())

[2019-08-30 17:24:06.237] [polyfem] [info] simplex_count: 	10331
[2019-08-30 17:24:06.237] [polyfem] [info] regular_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] regular_boundary_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] simple_singular_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] multi_singular_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] boundary_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] non_regular_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] undefined_count: 	0
[2019-08-30 17:24:06.237] [polyfem] [info] total count:	 10331
[2019-08-30 17:24:06.237] [polyfem] [info] Building isoparametric basis...
[2019-08-30 17:24:06.299] [polyfem] [info] Computing polygonal basis...
[2019-08-30 17:24:06.299] [polyfem] [info]  took 2.1912e-05s
[2019-08-30 17:24:06.302] [polyfem] [info] hmin: 0.4719

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.006010…

20


In [57]:
solver_ini.settings(settings)
solver_ini.solve()
p2, t2, d2 = solver_ini.get_sampled_solution()
m2 = np.linalg.norm(d2, axis=1)


mp.plot(p2+d2, t2, m2)

[2019-08-30 17:24:11.721] [polyfem] [info] simplex_count: 	10901
[2019-08-30 17:24:11.721] [polyfem] [info] regular_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] regular_boundary_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] simple_singular_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] multi_singular_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] boundary_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] non_regular_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] undefined_count: 	0
[2019-08-30 17:24:11.721] [polyfem] [info] total count:	 10901
[2019-08-30 17:24:11.721] [polyfem] [info] Building isoparametric basis...
[2019-08-30 17:24:11.786] [polyfem] [info] Computing polygonal basis...
[2019-08-30 17:24:11.786] [polyfem] [info]  took 1.3873e-05s
[2019-08-30 17:24:11.789] [polyfem] [info] hmin: 0.3017

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001647…

In [58]:

v_1, f_1 = extract_surface(p1+d1, t1)
igl.write_obj("1.obj", v_1, f_1)
v_1, f_1 = igl.read_triangle_mesh("1.obj")
v_2, f_2 = extract_surface(p2+d2, t2)
igl.write_obj("2.obj", v_2, f_2)
v_2, f_2 = igl.read_triangle_mesh("2.obj")
v_, f_ = extract_surface(p+d, t)
igl.write_obj("_.obj", v_, f_)
v_, f_ = igl.read_triangle_mesh("_.obj")
print(igl.hausdorff(v_1, f_1, v_2, f_2))
print(igl.hausdorff(v_, f_, v_2, f_2))

1.0662958348055822
1.0656775415256337


In [10]:
mp.plot(v_bad,igl.boundary_facets(f_bad))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0078250…

In [25]:
igl.boundary_facets(f_bad)

array([[  79,    0],
       [   0,  101],
       [   1,   79],
       ...,
       [1808, 1811],
       [1822, 1809],
       [1814, 1810]], dtype=int64)